# Gradient Boosting 

Possible lectures to this topic can be find in [1](http://benalexkeen.com/gradient-boosting-in-python-using-scikit-learn/), [2](https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html), [3](https://www.kaggle.com/beagle01/prediction-with-gradient-boosting-classifier), [4](https://en.wikipedia.org/wiki/Gradient_boosting) and [5](https://machinelearningmastery.com/gentle-introduction-xgboost-applied-machine-learning/)

Besides Random Forests, *Boosting* is another powerful approach to increase the predictive power of classical decision and regression tree models. The Boosting algorithm itself can strictly speaking neither learn nor predict anything since it is build kind of on top of some other (weak) algorithm. The Boosting algorithm is called a "meta algorithm". The Boosting approach can (as well as the bootstrapping approach), be applied, in principle, to any classification or regression algorithm but it turned out that tree models are especially suited. The accuracy of boosted trees turned out to be equivalent to Random Forests with respect and even often outperforms the latter (see for instance Caruana and Niculescu-Mizil (2008)(*An Empirical Comparison of Supervised Learning Algorithms*)). Hastie et al. (2009) call boosted decision trees the "best off-the-shelf classifier of the world" (Hastie et al. 2006 p.340). The mystic behind Boosting is in principal the same as for Random Forest models *-A bunch of weak learners which performs just slightly better than random guessing can be combined to make better predictions than one strong learner-*. Though, the process how these weak learners are created differs. 
Recapitulate, that during the creation of the Random Forest model we used the concept of Bagging. During Bagging we have grown a number of *M* trees where each was build on a random sample (allowing resampling) of the original dataset where the random sample had the same length as the original dataset but comprises only a randomly drawn subset of the total feature space. After we have created theses models, we let them make a majority vote to make our final decision. The quintessence is that each tree model is created independent from the outcomes of the other tree models. That is, the "shape" of the tree model is only influenced by the "shape" of the underlying data which in turn is only influenced by chance (*sampling with resampling*). The main difference in the creation of bagged trees using bootstrap aggregation and boosted trees using boosting is that we now replace the (random) resampling by some kind of *weighting* where we allocate the instances with weights and the weights of the *$n^{th}$* tree depends on the results returned by the previously created, $n^{th−1}$, tree model. Hence, different from the Random Forest approach where we created an ensemble of tree models in parallel, we now create the ensemble **in sequence**, where the set up of the actual tree is influenced by the outputs of all the previous tree models by altering the weights of the dataset, the tree model is build on. The point is, that by implementing these weights, we introduce some kind of learning where the creation of the *$n^{th}$* tree in the boosted model partly depends on the predictions the *$n^{th−1}$* model has made. Therewith, we replace the more or less "randomly-guided" creation of the single datasets during bootstrapping by a "guided" creation. The most prominent boosting algorithm is called *AdaBoost* (adaptive boosting) and was developed by Freund and Schapire (1996). The following discussion is based on the AdaBoost Boosting algorithm. The following illustration gives a visual insight into the boosting algorithm. 
<img src="images/boost.png" width="500" height="600">
Illustrates the boosing algorithm. The different base algorithms are each build on a weighted dataset where the weights of the single instances in the dataset depend on the results the previous base classifiers had made for these instances. If they have misclassified a instance, the weight for this instance will be increased in the next model while if the classification was correct, the weight is unaltered. The final decision making is achieved by a weighted vote of the base classifiers where the weights are determined  depending on the misclassification rate of the models. If a model has had a high classification accuracy, it will get a high weights while it gets a low weight if it has has a poor classification accuracy.  
Here the different base classifiers are each build on a weighted dataset where the weights of the single instances in the dataset depend on the results the previous base classifiers had made for these instances. If they have misclassified a instance, the weight for this instance will be increased in the next model while if the classification was correct, the weight remains unaltered. The final decision making is achieved by a weighted vote of the base classifiers where the weights are determined depending on the misclassification rates of the models. If a model has had a high classification accuracy, it will get a high weight while it gets a low weight if it has had a poor classification accuracy.

<img src="images/compare.png" width="500" height="600">

## Boosting Pseudocode
Initialize all weights to $w=1/n$ where $n$ is the number of instances in the dataset 
- while $t < T$ ($T$==number of models to be grown) do:
- Create a model and get the hypothesis $h_t(x_n)$ for all datapoints $x_n$ in the dataset
- Calculate the error $\epsilon$ of the training set summing over all datapoints $x_n$ in the training set with:

$ \epsilon_{t} = \frac{ \sum_{n=1}^{N}w^{(t)}_{n}*I(y_{n}\neq h_t(x_n))}{\sum_{n=1}^{N}w^{(t)}_{n}}$

where $I$(cond) returns 1 if $I$(cond) == True and 0 otherwise
- Compute $\alpha$ with:

$\alpha_t = log(\frac{1-\epsilon_t}{\epsilon_t})$

- Update the weights for the $N$ training instances in the next $(t+1)$ model with:

$w^{(t+1)}_n  =  w^{(t)}_n∗exp(\alpha_t∗I(y_n\neq h_t(x_n)))$

After the $T$ iterations, calculate the final output with:

$f(x)  =  sign(\sum_{t}^{T} \alpha_t∗h_t(x))$

In [1]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
# load data
dataset = loadtxt('data/pima-indians-diabetes.csv', delimiter=",")

In [3]:
# split data into X and y
X = dataset[:,0:8]
Y = dataset[:,8]

In [4]:
# split data into train and test sets
seed = 7
test_size = 0.33
X_train_sub, X_validation_sub, y_train_sub, y_validation_sub = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [5]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    gb = GradientBoostingClassifier(n_estimators=20, learning_rate = learning_rate, max_features=2, max_depth = 2, random_state = 0)
    gb.fit(X_train_sub, y_train_sub)
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb.score(X_train_sub, y_train_sub)))
    print("Accuracy score (validation): {0:.3f}".format(gb.score(X_validation_sub, y_validation_sub)))

Learning rate:  0.05
Accuracy score (training): 0.768
Accuracy score (validation): 0.740
Learning rate:  0.1
Accuracy score (training): 0.792
Accuracy score (validation): 0.748
Learning rate:  0.25
Accuracy score (training): 0.823
Accuracy score (validation): 0.768
Learning rate:  0.5
Accuracy score (training): 0.852
Accuracy score (validation): 0.756
Learning rate:  0.75
Accuracy score (training): 0.864
Accuracy score (validation): 0.756
Learning rate:  1
Accuracy score (training): 0.881
Accuracy score (validation): 0.736


## Using Xgboost

XGBoost is an optimized distributed gradient boosting library designed to be highly efficient, flexible and portable.

How to install:

`conda install -c conda-forge xgboost`

`conda install -c conda-forge/label/gcc7 xgboost`

`conda install -c conda-forge/label/cf201901 xgboost`

In [6]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train_sub, y_train_sub)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
              n_jobs=1, nthread=None, objective='binary:logistic',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1)

In [7]:
print(model)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
              n_jobs=1, nthread=None, objective='binary:logistic',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1)


Make predictions

In [8]:
# make predictions for test data
y_pred = model.predict(X_validation_sub)
predictions = [round(value) for value in y_pred]

In [9]:
# evaluate predictions
accuracy = accuracy_score(y_validation_sub, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 77.95%


## Otros ejemplos

In [11]:
# Author: Peter Prettenhofer <peter.prettenhofer@gmail.com>
#
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt

from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

# #############################################################################
# Load data
boston = datasets.load_boston()
X, y = shuffle(boston.data, boston.target, random_state=13)
X = X.astype(np.float32)
offset = int(X.shape[0] * 0.9)
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]

# #############################################################################
# Fit regression model
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)
mse = mean_squared_error(y_test, clf.predict(X_test))
print("MSE: %.4f" % mse)

# #############################################################################
# Plot training deviance

# compute test set deviance
test_score = np.zeros((params['n_estimators'],), dtype=np.float64)

for i, y_pred in enumerate(clf.staged_predict(X_test)):
    test_score[i] = clf.loss_(y_test, y_pred)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, clf.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')

# #############################################################################
# Plot feature importance
feature_importance = clf.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.subplot(1, 2, 2)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, boston.feature_names[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

MSE: 6.5234


<Figure size 1200x600 with 2 Axes>